In [2]:
import sys; sys.path.append('../'); sys.path.append('../hashing')
from hashing import LinearHash

import numpy as np

np.random.seed(88)

# Exploración mejores y peores casos LinearHash

Sea $th$, el tamaño de tabla hash en cada instancia.

In [3]:
size_max = int(1e4)
sizes = [i for i in range(10, size_max, size_max//4)]
print(sizes)
l_hashes = [LinearHash(size=s, update_size=True) for s in sizes]

[10, 2510, 5010, 7510]


## Secuencia de elementos ordenados

Se crean las siguientes variaciones de secuencias:

1. Secuencia ordenada creciente con paso de 1
2. Secuencia ordenada creciente con paso de 3
3. Secuencia ordenada creciente con paso de $\frac{th}{4}$
4. Secuencia ordenada creciente con paso de aleatorio entre 1 y $\frac{th}{4}$

5. Un cuarto creciente, otro decreciente y bis (bloques aleatorios)
6. 4 bloques crecientes independientes

Ademas para cada una de estas, existen dos tamaño: el tamaño de la tabla y 3 veces el tamaño de la tabla.

In [3]:
# Ordered Sequence
all_ordered = np.array([np.array(range(len(lh)), dtype=int)
                        for lh in l_hashes])
np.save('casos_lhash/ordered_step_1', all_ordered)

all_ordered_oversize = np.asarray(
    [np.array(range(len(lh) * 3)) for lh in l_hashes])
np.save('casos_lhash/ordered_step_1_oversize', all_ordered_oversize)

# Ordered Sequence with step=3
ordered_step3 = np.asarray(
    [np.array(range(0, len(lh), 3), dtype=int) for lh in l_hashes])
np.save('casos_lhash/ordered_step_3', ordered_step3)

ordered_step3_oversize = np.asarray(
    [np.array(range(0, len(lh) * 3, 3), dtype=int) for lh in l_hashes])
np.save('casos_lhash/ordered_step_3_oversize', ordered_step3_oversize)

# Ordered Sequence with step=table_size/4 
ordered_stepquarter = np.asarray(
    [np.array(range(0, len(lh), len(lh)//4), dtype=int) for lh in l_hashes])
np.save('casos_lhash/ordered_step_quarter', ordered_stepquarter)

ordered_stepquarter_oversize = np.asarray(
    [np.array(range(0, len(lh)*3, len(lh)//4), dtype=int) for lh in l_hashes])
np.save('casos_lhash/ordered_step_quarter_oversize', ordered_stepquarter_oversize)


# Ordered Sequence with random step
ordered_steprandom = np.asarray(
    [np.array(range(0, len(lh), np.random.randint(1, len(lh)//4)), dtype=int) for lh in l_hashes])
np.save('casos_lhash/ordered_step_random', ordered_steprandom)


ordered_steprandom_oversize = np.asarray(
    [np.array(range(0, len(lh)*3, np.random.randint(1, len(lh)//4)), dtype=int) for lh in l_hashes])
np.save('casos_lhash/ordered_step_random_oversize', ordered_steprandom_oversize)

In [4]:
def random_array(n, min_val, max_val, sort=False):
    """
    Funcion para generar A y secuencias aleatorias.
    :param n: numero de elementos que tendrá el arreglo
    :param max_val: maximo valor aleatorio
    :return: un arreglo de valores ordenados
    """
    arr = []
    counter = 0
    pos_vals = list(range(min_val, max_val))
    while len(arr) != n:
        value = np.random.choice(pos_vals)
        
        arr.append(value)
        
        idx = pos_vals.index(value)
        del pos_vals[idx]

    if sort:
        arr.sort()
    return np.array(arr)

def blocks_ordered(size, n_blocks, inc_dec=False, threshold=0.5):
    blocks = []
    
    ind = False
    for i in range(n_blocks):
        arr = random_array(n=size, min_val=0, max_val=size, sort=True)
        
        if inc_dec == 'random' and np.random.random() > threshold:
            arr = reversed(arr)
        
        elif inc_dec and ind:
            arr = reversed(arr)
            ind = not ind
            
        elif inc_dec and not ind:
            ind = not ind 
            
        blocks += arr
    
    return np.array(blocks)
        

In [16]:
# Creciente, decreciente
inc_dec = np.array([blocks_ordered(len(lh)//4, n_blocks=4, inc_dec=True) for lh in l_hashes])
np.save('casos_lhash/ordered_blocks_intercalado', inc_dec)

# Creciente, decreciente de manera aleatoria
inc_dec_rand =  np.array([blocks_ordered(len(lh)//4, n_blocks=4, inc_dec='random') for lh in l_hashes])
np.save('casos_lhash/ordered_blocks_intercalado_rand', inc_dec_rand)

# Bloques crecientes independientes
inc =  np.array([blocks_ordered(len(lh)//4, n_blocks=4) for lh in l_hashes])
np.save('casos_lhash/ordered_blocks_creciente', inc)


In [15]:
len(inc_dec[-1])

7508

## Secuencia de elementos repetidos

Se crean las siguientes variaciones de secuencias:

1. Secuencia de 3 elementos repetidos con step de 1
2. Secuencia de 3 elementos repetidos con step de 3
3. Secuencia de 3 elementos repetidos con step de $\frac{th}{4}$
4. Secuencia de $\frac{th}{4}$ elementos repetidos con step aleatorio entre 1 y $\frac{th}{4}$

Ademas para cada una de estas, existen dos tamaño: el tamaño de la tabla y 3 veces el tamaño de la tabla.

In [6]:
def repetead_seq(size, step, num_repeats=3):
    seqs = np.array([], 'object')
    i = 0
    while len(seqs) <= size:
        val = 0
        seq = np.array([], 'int')
        
        while len(seq) <= size:            
            seq = np.append(seq, [val] * num_repeats) 
            val += step
            
        seqs = np.append(seqs, seq)
        
        i+=1
    return seqs

In [19]:
repeated_3_step_1[-1].shape

(7512,)

In [20]:
############ 3 repeats sequence step 1 ############

repeated_3_step_1 = np.array([repetead_seq(len(lh), 1)
                           for lh in l_hashes])
np.save('casos_lhash/repeated_3_1', repeated_3_step_1)

repeated_3_step_1_oversize = np.asarray(
    [repetead_seq(len(lh) * 3, 1) for lh in l_hashes])
np.save('casos_lhash/repeated_3_1_oversize', repeated_3_step_1_oversize)




############ 3 repeats sequence step 1=3 ############

repeated_3_step_3 = np.asarray(
    [repetead_seq(len(lh), 3) for lh in l_hashes])
np.save('casos_lhash/repeated_3_3', repeated_3_step_3)

repeated_3_step_3_oversize = np.asarray(
    [repetead_seq(len(lh) * 3, 3) for lh in l_hashes])
np.save('casos_lhash/repeated_3_3_oversize', repeated_3_step_3_oversize)





############ repeated Sequence with step_=table_size/4 ############

repeated_3_step_quarter = np.asarray(
    [repetead_seq(len(lh), len(lh)//4) for lh in l_hashes])
np.save('casos_lhash/repeated_3_quarter', repeated_3_step_quarter)

repeated_3_step_quarter_oversize = np.asarray(
    [repetead_seq(len(lh) * 3, len(lh)//4) for lh in l_hashes])
np.save('casos_lhash/repeated_3_quarter_oversize', repeated_3_step_quarter_oversize)





############ repeated n/4 Sequence with step_=table_size/4 ############

repeated_quarter_step_1 = np.asarray(
    [repetead_seq(len(lh), 1, len(lh)//4) for lh in l_hashes])
np.save('casos_lhash/repeated_quarter_1', repeated_quarter_step_1)


repeated_quarter_step_1_oversize = np.asarray(
    [repetead_seq(len(lh) * 3, 1, len(lh)//4) for lh in l_hashes])
np.save('casos_lhash/repeated_quarter_1_oversize', repeated_quarter_step_1_oversize)





############ repeated n/4 Sequence with random step_ ############

repeated_quarter_step_random = np.asarray(
    [repetead_seq(len(lh), np.random.randint(1, len(lh)//4), len(lh)//4) for lh in l_hashes])
np.save('casos_lhash/repeated_quarter_random', repeated_quarter_step_random)


repeated_quarter_step_random_oversize = np.asarray(
    [repetead_seq(len(lh) * 3, np.random.randint(1, len(lh)//4), len(lh)//4) for lh in l_hashes])
np.save('casos_lhash/repeated_quarter_random_oversize', repeated_quarter_step_random_oversize)

## Secuencia de elementos aleatorias

Se crean las siguientes variaciones de secuencias:

1. Secuencias aleatorias

In [ ]:
random_seq_n = np.array([random_array(n=len(lh), min_val=0, max_val=len(lh) * 2) for lh in l_hashes])
np.save('casos_lhash/random_seq_n', random_seq_n)

random_seq_n_oversize = np.array([random_array(n=len(lh) * 3, min_val=0, max_val=len(lh) * 3) for lh in l_hashes])
np.save('casos_lhash/random_seq_n_oversize', random_seq_n)